### 資料、套件導入

In [1]:
import os
import pandas as pd
import re 

In [2]:
path = '../data/0414/review_dataset.csv'
df = pd.read_csv(path)
df = df[:1272]
df

,reviews,value,comfort,location,cleanliness,service,facilities
0,價格合理 舒適的房間 老闆娘人很好還會自己做早餐給旅客 重點是早餐吃到飽,1.0,1.0,0.0,0.0,1.0,0.0
1,內部房間滿乾淨的，還有大場地，可以讓團體來使用，我覺得很棒,0.0,0.0,0.0,1.0,0.0,1.0
2,隔音還可以，房間不小，但美中不足沒有乾濕分離,0.0,1.0,0.0,0.0,0.0,0.0
3,房子設計的很棒 房間採光很好 大廳挑高氣派 房價合理 台東必住民宿,1.0,1.0,0.0,0.0,0.0,1.0
4,Cp值高 乾淨舒適空間大 樓下有免費吐司和咖啡 老闆回復速度快,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
1267,港式飲茶餐廳口味很棒，環境乾淨，機車汽車都有停車位，位於高鐵附近，挺適合宴客的。,0.0,0.0,1.0,1.0,1.0,1.0
1268,場地氣派、丁香魚酥脆、服務親切 蠟味蘿蔔糕份量多一些更好、牛肉粥好吃,0.0,0.0,0.0,0.0,1.0,0.0
1269,交通方便，地下室停車場良好，菜色好，空間設計好，真可說是一流的飯店。,0.0,0.0,1.0,0.0,1.0,1.0
1270,地點佳，離逢甲夜市很近，老闆娘很親切服務，房間夠大舒適，浴室也很乾淨,0.0,1.0,1.0,1.0,1.0,0.0


### 移除重複值、空值

In [3]:
#印出重複資料
df[df.duplicated()]

,reviews,value,comfort,location,cleanliness,service,facilities


In [4]:
#移除重複值
print(df.shape)
df = df.drop_duplicates() #移除
print(df.shape)

(1272, 7)
(1272, 7)


In [5]:
#印出空值資料
df[df.isnull().T.any()]

,reviews,value,comfort,location,cleanliness,service,facilities


### 拆分數據集

In [10]:
train_df = df[:int(df.shape[0]*(2/3))]
test_df = df[int(df.shape[0]*(2/3)):]
print("訓練集：",train_df.shape)
print("測試集：",test_df.shape)
test_df = test_df.reset_index(drop=True) #重新index

訓練集： (840, 7)
測試集： (420, 7)


### 資料前處理：移除標點符號、空格

In [6]:
def review_preprocessing(sentence):
    #sentence = '很乾淨，停車方便， 住的那天房價很便宜， 房間和外觀都超可愛\t'
    ## 把標點符號、空格拿掉
    pattern = re.compile(r'[^\w]') #匹配字母數字及下劃線
    sentence = re.sub(pattern, '', sentence)
    return sentence

In [7]:
def review_preprocessing_run(df):
    for i, row in df.iterrows():
        ifor_val = review_preprocessing(row['reviews']) #資料處理
        df.at[i,'reviews'] = ifor_val #更新資料
    return df

In [8]:
#train_df = review_preprocessing_run(train_df)
#test_df = review_preprocessing_run(test_df)
df = review_preprocessing_run(df)

### 中文分詞+詞性標注

In [9]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
import gzip, bz2
from datetime import datetime, timedelta

In [10]:
POS = True #是否也要一同執行詞性標記
KEEP_PUNCTUATION_POS = False

In [11]:
device = -1 # 0=GPU, -1=CPU
level = 3 #level: 3=most accurate, but model size largest
print("Initializing ws driver", flush=True) #flush 實現動態Loading
ws_driver  = CkipWordSegmenter(device=device, level=level)
if POS:
    print("Done initializing POS driver", flush=True)
    pos_driver = CkipPosTagger(device=device, level=level)
    print("Initializing pos driver", flush=True)

Initializing ws driver
Done initializing POS driver
Initializing pos driver


In [12]:
def ws_pos_review(sentence):
    ws  = ws_driver([sentence])
    pos = pos_driver(ws) 

    # 打包結果
    assert len(ws[0]) == len(pos[0]) #檢查 ws 和 pos是否長度一致
    res = []
    for word_ws, word_pos in zip(ws[0], pos[0]):
        res.append(f"{word_ws}({word_pos})")
    
    return ",".join(res)

In [13]:
def ws_pos_review_run(df):
    for i, row in df.iterrows():
        ws_pos_list = ws_pos_review(row['reviews']) #分詞+詞性標注
        df.at[i,'ws_pos_reviews'] = ws_pos_list #更新資料
        print("目前進度：" + str(i))
    return df

In [14]:
#train_df_ws = ws_pos_review_run(train_df)
#test_df_ws = ws_pos_review_run(test_df)
df_ws = ws_pos_review_run(df)

Inference: 100%|██████████| 1/1 [00:00<00:00, 14.03it/s]


目前進度：0


Inference: 100%|██████████| 1/1 [00:00<00:00, 15.32it/s]


目前進度：1


Inference: 100%|██████████| 1/1 [00:00<00:00, 18.89it/s]


...

目前進度：1269


Inference: 100%|██████████| 1/1 [00:00<00:00, 16.96it/s]


目前進度：1270


Inference: 100%|██████████| 1/1 [00:00<00:00, 19.16it/s]

目前進度：1271


In [15]:
#train_df_ws
df_ws

,reviews,value,comfort,location,cleanliness,service,facilities,ws_pos_reviews
0,價格合理舒適的房間老闆娘人很好還會自己做早餐給旅客重點是早餐吃到飽,1.0,1.0,0.0,0.0,1.0,0.0,"價格(Na),合理(VH),舒適(VH),的(DE),房間(Nc),老闆娘(Na),人(Na..."
1,內部房間滿乾淨的還有大場地可以讓團體來使用我覺得很棒,0.0,0.0,0.0,1.0,0.0,1.0,"內部(Ncd),房間(Nc),滿(Dfa),乾淨(VH),的(DE),還(D),有(V_2)..."
2,隔音還可以房間不小但美中不足沒有乾濕分離,0.0,1.0,0.0,0.0,0.0,0.0,"隔音(A),還可以(D),房間(Nc),不(D),小(VH),但(Cbb),美中不足(VH)..."
3,房子設計的很棒房間採光很好大廳挑高氣派房價合理台東必住民宿,1.0,1.0,0.0,0.0,0.0,1.0,"房子(Na),設計(VC),的(DE),很(Dfa),棒(VH),房間(Nc),採光(Na)..."
4,Cp值高乾淨舒適空間大樓下有免費吐司和咖啡老闆回復速度快,1.0,1.0,0.0,0.0,1.0,0.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc..."
...,...,...,...,...,...,...,...,...
1267,港式飲茶餐廳口味很棒環境乾淨機車汽車都有停車位位於高鐵附近挺適合宴客的,0.0,0.0,1.0,1.0,1.0,1.0,"港式(Na),飲茶(VA),餐廳(Nc),口味(Na),很(Dfa),棒(VH),環境(Na..."
1268,場地氣派丁香魚酥脆服務親切蠟味蘿蔔糕份量多一些更好牛肉粥好吃,0.0,0.0,0.0,0.0,1.0,0.0,"場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(..."
1269,交通方便地下室停車場良好菜色好空間設計好真可說是一流的飯店,0.0,0.0,1.0,0.0,1.0,1.0,"交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(..."
1270,地點佳離逢甲夜市很近老闆娘很親切服務房間夠大舒適浴室也很乾淨,0.0,1.0,1.0,1.0,1.0,0.0,"地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),很(Dfa),近(VH),老..."


In [16]:
#test_df_ws

### 移除停用詞

In [19]:
def remove_stop(df):
    df_2 = df #將分詞、詞性標註的結果存到另一個df裡 
    stopwords_path='../data/stopwords_TW2.txt' #使用哈工大停用詞表
    # 設定停用詞
    print('start read stopwords data.')
    stopwords = []
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        for line in f:
            if len(line)>0:
                stopwords.append(line.strip())

    rawlist = df_2['ws_pos_reviews'].tolist() 

    for idx,element in enumerate(rawlist):
        filter_list = []
        rawlist2 = element.split(",") #轉成 list
        for val in rawlist2: #去比較list裡面每一個元素，是否屬於stopwords
            extract_str = re.search('(.*?)\(', val).group(1) #擷取"("前面的字串
            if(extract_str not in stopwords): # 如果屬於stopwords，就append到新的filter_list
                filter_list.append(val)
        df_2.at[idx,'filtered'] = ",".join(filter_list) #更新df_2資料
    return df_2

In [20]:
#train_df = remove_stop(train_df_ws)
#test_df = remove_stop(test_df_ws)
df = remove_stop(df_ws)

start read stopwords data.


### 存檔

In [21]:
#train_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_train.csv', encoding='utf_8_sig', index=False)
#test_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_test.csv', encoding='utf_8_sig', index=False)
df.to_csv('../data/0414/review_data(seg+pos+stopwords).csv', encoding='utf_8_sig', index=False)

### 擷取名詞和動詞、外語、介詞

In [40]:
def getword(sentence):
    #sentence = '價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),很(Dfa),好(VH),還(D),會(D),做(VC),早餐(Na),旅客(Na),重點(Na),早餐(Na),吃到飽(VC)'
    res=[]
    #pattern = r"\b[^,]*\([N,V,F,P][^,]*"
    pattern = r"\b[^,]*\([N,A,V][^,]*"
    matches = re.finditer(pattern, sentence)
    for matchNum, match in enumerate(matches, start=1):
        res.append(match.group()) #放進list
    #將list轉成string
    filter_str = ','.join(res)
    return filter_str
def getword2(sentence): #不要Ne..=>一些數字
    #sentence = '價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),很(Dfa),好(VH),還(D),會(D),做(VC),早餐(Na),旅客(Na),重點(Na),早餐(Na),吃到飽(VC)'
    res=[]
    #pattern = r"\b[^,]*\([N,V,F,P][^,]*"
    pattern = r"\b[^,]*\([(N][^e][^,]*"
    matches = re.finditer(pattern, sentence)
    for matchNum, match in enumerate(matches, start=1):
        res.append(match.group()) #放進list
    pattern = r"\b[^,]*\([A,V][^,]*"
    for matchNum, match in enumerate(matches, start=1):
        res.append(match.group()) #放進list
    #將list轉成string
    filter_str = ','.join(res)
    return filter_str

In [23]:
#train_df['filtered_word'] = train_df.apply(lambda x: getword(x['filtered']),axis=1)
#test_df['filtered_word'] = test_df.apply(lambda x: getword(x['filtered']),axis=1)
df['filtered_word'] = df.apply(lambda x: getword(x['filtered']),axis=1)

In [35]:
#test_df
df

,reviews,value,comfort,location,cleanliness,service,facilities,ws_pos_reviews,filtered,filtered_word
0,價格合理舒適的房間老闆娘人很好還會自己做早餐給旅客重點是早餐吃到飽,1.0,1.0,0.0,0.0,1.0,0.0,"價格(Na),合理(VH),舒適(VH),的(DE),房間(Nc),老闆娘(Na),人(Na...","價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),好(VH...","價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),好(VH..."
1,內部房間滿乾淨的還有大場地可以讓團體來使用我覺得很棒,0.0,0.0,0.0,1.0,0.0,1.0,"內部(Ncd),房間(Nc),滿(Dfa),乾淨(VH),的(DE),還(D),有(V_2)...","內部(Ncd),房間(Nc),乾淨(VH),場地(Na),團體(Na),使用(VC),覺得(...","內部(Ncd),房間(Nc),乾淨(VH),場地(Na),團體(Na),使用(VC),覺得(..."
2,隔音還可以房間不小但美中不足沒有乾濕分離,0.0,1.0,0.0,0.0,0.0,0.0,"隔音(A),還可以(D),房間(Nc),不(D),小(VH),但(Cbb),美中不足(VH)...","隔音(A),還可以(D),房間(Nc),小(VH),美中不足(VH),乾(VH),濕(VH)...","隔音(A),房間(Nc),小(VH),美中不足(VH),乾(VH),濕(VH),分離(VHC)"
3,房子設計的很棒房間採光很好大廳挑高氣派房價合理台東必住民宿,1.0,1.0,0.0,0.0,0.0,1.0,"房子(Na),設計(VC),的(DE),很(Dfa),棒(VH),房間(Nc),採光(Na)...","房子(Na),設計(VC),棒(VH),房間(Nc),採光(Na),好(VH),大廳(Nc)...","房子(Na),設計(VC),棒(VH),房間(Nc),採光(Na),好(VH),大廳(Nc)..."
4,Cp值高乾淨舒適空間大樓下有免費吐司和咖啡老闆回復速度快,1.0,1.0,0.0,0.0,1.0,0.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc...","Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc...","高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Ncd),免費(VH..."
...,...,...,...,...,...,...,...,...,...,...
1267,港式飲茶餐廳口味很棒環境乾淨機車汽車都有停車位位於高鐵附近挺適合宴客的,0.0,0.0,1.0,1.0,1.0,1.0,"港式(Na),飲茶(VA),餐廳(Nc),口味(Na),很(Dfa),棒(VH),環境(Na...","港式(Na),飲茶(VA),餐廳(Nc),口味(Na),棒(VH),環境(Na),乾淨(VH...","港式(Na),飲茶(VA),餐廳(Nc),口味(Na),棒(VH),環境(Na),乾淨(VH..."
1268,場地氣派丁香魚酥脆服務親切蠟味蘿蔔糕份量多一些更好牛肉粥好吃,0.0,0.0,0.0,0.0,1.0,0.0,"場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(...","場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(...","場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(..."
1269,交通方便地下室停車場良好菜色好空間設計好真可說是一流的飯店,0.0,0.0,1.0,0.0,1.0,1.0,"交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(...","交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(...","交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(..."
1270,地點佳離逢甲夜市很近老闆娘很親切服務房間夠大舒適浴室也很乾淨,0.0,1.0,1.0,1.0,1.0,0.0,"地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),很(Dfa),近(VH),老...","地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),老闆娘(Na),親切(VH)...","地點(Na),佳(VH),逢甲(Nb),夜市(Nc),老闆娘(Na),親切(VH),服務(V..."


In [25]:
#train_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_train_n+v+f+p.csv', encoding='utf_8_sig', index=False)
#test_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_test_n+v+f+p.csv', encoding='utf_8_sig', index=False)
df.to_csv('../data/0414/review_data(seg+pos+stopwords)_n+v+f+p.csv', encoding='utf_8_sig', index=False)

In [41]:
path = '../data/0414/review_data(seg+pos+stopwords).csv'
df = pd.read_csv(path)
df

,reviews,value,comfort,location,cleanliness,service,facilities,ws_pos_reviews,filtered
0,價格合理舒適的房間老闆娘人很好還會自己做早餐給旅客重點是早餐吃到飽,1.0,1.0,0.0,0.0,1.0,0.0,"價格(Na),合理(VH),舒適(VH),的(DE),房間(Nc),老闆娘(Na),人(Na...","價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),好(VH..."
1,內部房間滿乾淨的還有大場地可以讓團體來使用我覺得很棒,0.0,0.0,0.0,1.0,0.0,1.0,"內部(Ncd),房間(Nc),滿(Dfa),乾淨(VH),的(DE),還(D),有(V_2)...","內部(Ncd),房間(Nc),乾淨(VH),場地(Na),團體(Na),使用(VC),覺得(..."
2,隔音還可以房間不小但美中不足沒有乾濕分離,0.0,1.0,0.0,0.0,0.0,0.0,"隔音(A),還可以(D),房間(Nc),不(D),小(VH),但(Cbb),美中不足(VH)...","隔音(A),還可以(D),房間(Nc),小(VH),美中不足(VH),乾(VH),濕(VH)..."
3,房子設計的很棒房間採光很好大廳挑高氣派房價合理台東必住民宿,1.0,1.0,0.0,0.0,0.0,1.0,"房子(Na),設計(VC),的(DE),很(Dfa),棒(VH),房間(Nc),採光(Na)...","房子(Na),設計(VC),棒(VH),房間(Nc),採光(Na),好(VH),大廳(Nc)..."
4,Cp值高乾淨舒適空間大樓下有免費吐司和咖啡老闆回復速度快,1.0,1.0,0.0,0.0,1.0,0.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc...","Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc..."
...,...,...,...,...,...,...,...,...,...
1267,港式飲茶餐廳口味很棒環境乾淨機車汽車都有停車位位於高鐵附近挺適合宴客的,0.0,0.0,1.0,1.0,1.0,1.0,"港式(Na),飲茶(VA),餐廳(Nc),口味(Na),很(Dfa),棒(VH),環境(Na...","港式(Na),飲茶(VA),餐廳(Nc),口味(Na),棒(VH),環境(Na),乾淨(VH..."
1268,場地氣派丁香魚酥脆服務親切蠟味蘿蔔糕份量多一些更好牛肉粥好吃,0.0,0.0,0.0,0.0,1.0,0.0,"場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(...","場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(..."
1269,交通方便地下室停車場良好菜色好空間設計好真可說是一流的飯店,0.0,0.0,1.0,0.0,1.0,1.0,"交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(...","交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(..."
1270,地點佳離逢甲夜市很近老闆娘很親切服務房間夠大舒適浴室也很乾淨,0.0,1.0,1.0,1.0,1.0,0.0,"地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),很(Dfa),近(VH),老...","地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),老闆娘(Na),親切(VH)..."


In [42]:
df['filtered_word'] = df.apply(lambda x: getword2(x['filtered']),axis=1)

In [43]:
df.to_csv('../data/0414/review_data(seg+pos+stopwords)_a+n+v.csv', encoding='utf_8_sig', index=False)